# BAIT509 Group Project
primary goal, target variable: da001, how does people feel about their health.

## 1. Data Preprocessing

## 1. 1 fetching data from SQL server

In [1]:
# create sql connection
from sqlalchemy import create_engine
import pandas as pd
import json
import urllib.parse
import psycopg2
import os 
with open("credentials.json") as file:
    login = json.load(file)
    
username = login['user']
password = urllib.parse.quote(login['password'])
host = login['host']
port = login['port']

engine = create_engine("postgresql://{}:{}@{}:{}/postgres".format(username,password,host,port))

In [2]:
# merge data
if 'df.csv' not in os.listdir():
    with engine.connect() as cnn:
        """
        With any proxy parameter == TRUE, this questionnair is answered by others, so we will drop them all
        """
        df = pd.read_sql("""
                            SELECT 
                                a."UID",
                                a."householdUID", 
                                a."communityUID",
                                proxy_2,
                                ba001,
                                ba009,
                                ba011,
                                ba014,
                                ba015,
                                ba016,
                                ba017,
                                proxy,
                                proxy_5,
                                da001,
                                da002_1_,
                                da002_2_,
                                da002_3_,
                                da002_4_,
                                da002_5_,
                                da002_6_,
                                da002_7_,
                                da002_8_,
                                da002_9_,
                                da002_10_,
                                da002_11_,
                                da002_12_,
                                da002_13_,
                                da002_14_,
                                da013,
                                da019,
                                da022,
                                da025,
                                da027,
                                da029,
                                da030,
                                da031,
                                da038_s1,
                                da038_s2,
                                da038_s3,
                                da038_s4,
                                da038_s5,
                                da038_s6,
                                da038_s7,
                                da038_s8,
                                da038_s9,
                                da040,
                                da050_1,
                                da051,
                                db001,
                                db003,
                                db005,
                                db007,
                                db009,
                                db011,
                                db012,
                                db014,
                                db016,
                                db018,
                                db020,
                                db022,
                                fa001,
                                fc006,
                                fc008
                            from charles.work_retirement a
                            join charles.health_status_and_functioning using("UID")
                            join charles.demographic_background using("UID")
                                        """, con = cnn)
        df.to_csv('df.csv')
else:
    pass

## 1.2. Drop unrelative data

In [3]:
# drop survey done by others
df = pd.read_csv('df.csv', index_col=0)
proxy_var = ['proxy','proxy_2','proxy_5']
mask = df[proxy_var].apply(lambda x: x.str.contains('1 Yes')).any(axis=1)
df = df[~mask].drop(labels=proxy_var, axis=1)

* Now based on data types, will will process different proprecessing for them:
    * imputer
    * binary: keep it with 0 or 1
    * categorical:
      * if with meaning, use ordinal encode, positive impact with larger number
      * if not, use one-hot encode
    * numerical:
      * scalar


### 1.3. Transform data from string to numeric data

In [4]:
df.iloc[:,3:].apply(lambda x: x.unique())

ba001                                       [1 Male, 2 Female]
ba009        [2 Non-Agricultural HuKou, 3 Unified Residence...
ba011        [1 Married with Spouse Present, 4 Divorced, 5 ...
ba014        [1 One Month, Participate in the Endowment Ins...
ba015        [2 Worker's Basic Endownment Insurace, 3 Socia...
ba016        [2 No Need to Pay, 1 Cost Per Year, 3 No Socia...
ba017                      [5.0, 3.0, 1.0, nan, 4.0, 2.0, 6.0]
da001        [1 Very Good, 3 Fair, 4 Poor, 2 Good, 5 Very P...
da002_1_     [nan, 3 Same, 1 Better, 2 Worse, 99 Never Had ...
da002_2_     [3 Same, nan, 1 Better, 2 Worse, 99 Never Had ...
da002_3_     [nan, 2 Worse, 3 Same, 99 Never Had the Diseas...
da002_4_     [nan, 1 Better, 3 Same, 99 Never Had the Disea...
da002_5_     [nan, 99 Never Had the Disease, 3 Same, 1 Bett...
da002_6_     [nan, 99 Never Had the Disease, 3 Same, 1 Bett...
da002_7_     [nan, 3 Same, 2 Worse, 1 Better, 99 Never Had ...
da002_8_     [nan, 1 Better, 3 Same, 99 Never Had the D

### 1.4. Fill NaN and Error values
| Column    | Non-Null Count | Data Type | Fill Strategy                                |
|-----------|----------------|-----------|----------------------------------------------|
| ba015     | 14982          | object    | fill 0                                       |
| ba017     | 16764          | float64   | fill 0                                       |
| da002_1_  | 5896           | object    | fill 99 then replace to 0, same for all da002_x_ |
| da002_2_  | 3457           | object    |                                              |
| da002_3_  | 1965           | object    |                                              |
| da002_4_  | 308            | object    |                                              |
| da002_5_  | 2041           | object    |                                              |
| da002_6_  | 1068           | object    |                                              |
| da002_7_  | 2992           | object    |                                              |
| da002_8_  | 982            | object    |                                              |
| da002_9_  | 1482           | object    |                                              |
| da002_10_ | 4941           | object    |                                              |
| da002_11_ | 291            | object    |                                              |
| da002_12_ | 12             | object    |                                              |
| da002_13_ | 285            | object    |                                              |
| da002_14_ | 6085           | object    |                                              |
| da013     | 17472          | object    | fill 2                                       |
| da019     | 17472          | object    | fill 2                                       |
| da022     | 17472          | object    | fill 2                                       |
| da025     | 17472          | object    | fill 2                                       |
| da027     | 17559          | object    | fill 1                                       |
| da029     | 17559          | object    | fill 3                                       |
| da030     | 17558          | float64   | fill mean                                    |
| da031     | 17558          | float64   | fill mean                                    |
| da038_s1  | 17555          | object    | fill 0 for all da038                         |
| da038_s2  | 17555          | object    |                                              |
| da038_s3  | 17555          | object    |                                              |
| da038_s4  | 17555          | object    |                                              |
| da038_s5  | 17555          | object    |                                              |
| da038_s6  | 17555          | object    |                                              |
| da038_s7  | 17555          | object    |                                              |
| da038_s8  | 17555          | object    |                                              |
| da038_s9  | 17555          | object    |                                              |
| da040     | 17555          | object    |                                              |
| da049_1   | 632            | float64   | delete bad data                              |
| da050_1   | 4494           | float64   | fill 0                                       |
| da051     | 17555          | object    | fill 3                                       |
| db001     | 17555          | object    | fill 1 for all db0XX                         |
| db003     | 17555          | object    |                                              |
| db005     | 17555          | object    |                                              |
| db007     | 17555          | object    |                                              |
| db009     | 17555          | object    |                                              |
| db011     | 17555          | object    |                                              |
| db012     | 17555          | object    |                                              |
| db014     | 17555          | object    |                                              |
| db016     | 17555          | object    |                                              |
| db018     | 17555          | object    |                                              |
| db020     | 17555          | object    |                                              |
| db022     | 17555          | object    |                                              |
| fa001     | 17562          | object    |                                              |
| fc006     | 397            | object    | fill 2                                       |
| fc008     | 344            | object    | fill 2                                       |


In [5]:
# for different columns we will apply different strategies
fillna0 = ['ba015','ba017']+['da038_s'+str(i) for i in range(1,10)]+['da050_1']
fillna1 = ['da027']+['db00'+str(i*2+1) for i in range(1,5)]+['db011']+['db0'+str(i*2) for i in range(6,12)]
fillna2 = ['da013','da019','da022','da025','fc006','fc008']
fillna3 = ['da029','da051']
fillna99_0 = ['da002_'+str(i)+'_' for i in range(1,15)]
fillna_mean = ['da030','da031']
dropna = ['fa001','da040']

In [6]:
# firstly, for all elements we only want the first number of all string
df = df.applymap(lambda x: x.split()[0] if isinstance(x, str) else x) 
df = df.applymap(lambda x: float(x) if isinstance(x,str) and len(x)<4 else x)

In [7]:
# we still have some non numeric rows, we will deal with it separately
df.ba014.unique()

# since Participate means the man haven't meet the pension age, so we manually assign it with 6 with means better than no pension but worse than have received pension now,
df.ba014 = df.ba014.replace('Participate',6)

In [8]:
# now we will fill NaN with rules we have established before
fill_na_list = {i:0 for i in fillna0} | {i:1 for i in fillna1} | {i:2 for i in fillna2} | {i:3 for i in fillna3} | {i:99 for i in fillna99_0} | {i:df[i].mean() for i in fillna_mean}
df.fillna(value=fill_na_list, inplace = True)
df = df.dropna(subset = dropna)

In [9]:
# check if null
df.isnull().sum().sum()

0

In [10]:
# let us to the check again 

df.iloc[:,3:].apply(lambda x: x.unique())

ba001                                               [1.0, 2.0]
ba009                                     [2.0, 3.0, 1.0, 4.0]
ba011                           [1.0, 4.0, 5.0, 6.0, 2.0, 3.0]
ba014                      [1.0, 6.0, 2.0, 7.0, 4.0, 3.0, 5.0]
ba015                      [2.0, 3.0, 1.0, 0.0, 4.0, 5.0, 6.0]
ba016                                          [2.0, 1.0, 3.0]
ba017                      [5.0, 3.0, 1.0, 0.0, 4.0, 2.0, 6.0]
da001                         [1.0, 3.0, 4.0, 2.0, 5.0, 997.0]
da002_1_                                 [99.0, 3.0, 1.0, 2.0]
da002_2_                                 [3.0, 99.0, 1.0, 2.0]
da002_3_                                 [99.0, 2.0, 3.0, 1.0]
da002_4_                                 [99.0, 1.0, 3.0, 2.0]
da002_5_                                 [99.0, 3.0, 1.0, 2.0]
da002_6_                                 [99.0, 3.0, 1.0, 2.0]
da002_7_                                 [99.0, 3.0, 2.0, 1.0]
da002_8_                                 [99.0, 1.0, 3.

We still have some columns contain 997 which means don't know, we will fill it with the average. While some others contain 99 which means they never have this disease, we will fill it with 0;

In [11]:
df[fillna99_0] = df[fillna99_0].replace(99,0)

df['da001'] = df['da001'].replace(997,df.da001[df.da001!=997].mean())
df['da029'] = df['da029'].replace(997,df.da029[df.da029!=997].mean())

In [12]:
# final check
df.describe()

,UID,householdUID,communityUID,ba001,ba009,ba011,ba014,ba015,ba016,ba017,...,db011,db012,db014,db016,db018,db020,db022,fa001,fc006,fc008
count,1.755500e+04,1.755500e+04,1.755500e+04,17555.000000,17555.000000,17555.000000,17555.000000,17555.000000,17555.000000,17555.000000,...,17555.000000,17555.000000,17555.000000,17555.000000,17555.000000,17555.000000,17555.000000,17555.000000,17555.000000,17555.000000
mean,1.764808e+11,1.764808e+09,1.764807e+06,1.535118,1.356878,1.668812,3.778240,3.187696,1.254457,3.215038,...,1.052008,1.211905,1.161265,1.128169,1.154657,1.075306,1.161435,1.506636,1.997721,1.989405
std,1.061556e+11,1.061556e+09,1.061556e+06,0.498779,0.663385,1.395856,2.518788,1.708115,0.529754,1.350191,...,0.278089,0.621466,0.581143,0.535916,0.599604,0.362165,0.591348,0.499970,0.047681,0.102389
min,1.010410e+10,1.010410e+08,1.010410e+05,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,7.400421e+10,7.400421e+08,7.400420e+05,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000
50%,1.624311e+11,1.624311e+09,1.624311e+06,2.000000,1.000000,1.000000,4.000000,4.000000,1.000000,4.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,2.000000
75%,2.811043e+11,2.811043e+09,2.811043e+06,2.000000,2.000000,1.000000,6.000000,4.000000,1.000000,4.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,2.000000
max,3.477633e+11,3.477633e+09,3.477633e+06,2.000000,4.000000,6.000000,7.000000,6.000000,3.000000,6.000000,...,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,2.000000,2.000000,2.000000


In [13]:
df.iloc[:,3:].apply(lambda x: x.unique())

ba001                                               [1.0, 2.0]
ba009                                     [2.0, 3.0, 1.0, 4.0]
ba011                           [1.0, 4.0, 5.0, 6.0, 2.0, 3.0]
ba014                      [1.0, 6.0, 2.0, 7.0, 4.0, 3.0, 5.0]
ba015                      [2.0, 3.0, 1.0, 0.0, 4.0, 5.0, 6.0]
ba016                                          [2.0, 1.0, 3.0]
ba017                      [5.0, 3.0, 1.0, 0.0, 4.0, 2.0, 6.0]
da001            [1.0, 3.0, 4.0, 2.0, 5.0, 2.9472603911283426]
da002_1_                                  [0.0, 3.0, 1.0, 2.0]
da002_2_                                  [3.0, 0.0, 1.0, 2.0]
da002_3_                                  [0.0, 2.0, 3.0, 1.0]
da002_4_                                  [0.0, 1.0, 3.0, 2.0]
da002_5_                                  [0.0, 3.0, 1.0, 2.0]
da002_6_                                  [0.0, 3.0, 1.0, 2.0]
da002_7_                                  [0.0, 3.0, 2.0, 1.0]
da002_8_                                  [0.0, 1.0, 3.

Now we have transfromed all of them data into numeric, but we still need to scalar some *real* numeric data 

## 2. Train and Test set split


In [17]:
# import a custom function
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
import altair as alt
import warnings
warnings.filterwarnings('ignore')
import os
import sys
sys.path.append(os.path.join(os.path.abspath("."), "code"))
from code.plotting_functions import *
from code.utils import *

In [ ]:
to_be_scalar = ['da030','da031','da050_1']